<a href="https://colab.research.google.com/github/LilySu/BetterBusinessByReview/blob/master/Word_Phrase_Endpoint_Refactoring3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scattertext
!pip install -U spacy
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 6.8MB 3.6MB/s 
     |████████████████████████████████| 10.4MB 3.1MB/s 
     |████████████████████████████████| 122kB 14.6MB/s 
     |████████████████████████████████| 2.2MB 35.3MB/s 
     |████████████████████████████████| 3.7MB 39.2MB/s 
  Found existing installation: preshed 2.0.1
    Uninstalling preshed-2.0.1:
      Successfully uninstalled preshed-2.0.1
  Found existing installation: blis 0.2.4
    Uninstalling blis-0.2.4:
      Successfully uninstalled blis-0.2.4
  Found existing installation: thinc 7.0.8
    Uninstalling thinc-7.0.8:
      Successfully uninstalled thinc-7.0.8
  Found existing installation: spacy 2.1.9
    Uninstalling spacy-2.1.9:
      Successfully uninstalled spacy-2.1.9
     |████████████████████████████████| 12.0MB 1.8MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-cp36-none-any.whl size=12011741 sha256=160e81703fca82d278c5d1ff5d2563211a70eb61466b9f3a1a7d2b3b91d8f55f
  Stored in directory: /tmp/

In [2]:
import json
import warnings
import pandas as pd
import numpy as np
# import spacy
# import scattertext as st
from lxml import html
from requests import Session
from concurrent.futures import ThreadPoolExecutor as Executor
import requests
import re
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_colwidth', 1000)
# nlp = spacy.load("en_core_web_sm")#if you run into problems here, 'Restart Runtime' and run all, it might fix things.
base_url = "https://www.yelp.com/biz/" 
api_url = "/review_feed?sort_by=date_desc&start="
bid = 'JqQdugxFSmGw797n-qxsMw'
# bid = 'Rc1lxc5lSKJYd162JHNMfQ'

class Scraper():
    def __init__(self):
        self.data = pd.DataFrame()

    def get_data(self, n, bid=bid):
        with Session() as s:
            with s.get(base_url+bid+api_url+str(n*20)) as resp: #makes an http get request to given url and returns response as json
                r = json.loads(resp.content) #converts json response into a dictionary
                _html = html.fromstring(r['review_list']) #loads from dictionary

                dates = _html.xpath("//div[@class='review-content']/descendant::span[@class='rating-qualifier']/text()")
                reviews = [el.text for el in _html.xpath("//div[@class='review-content']/p")]
                ratings = _html.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")

                df = pd.DataFrame([dates, reviews, ratings]).T

                self.data = pd.concat([self.data,df])

    def scrape(self): #makes it faster
        # multithreaded looping
        with Executor(max_workers=40) as e:
            list(e.map(self.get_data, range(10)))

s = Scraper()
s.scrape()
df = s.data
df = df.dropna()

df['word_segments_unpacked'] = df[1].apply(lambda x: x[1:-1].split(' '))

df['word_segments_unpacked'] = df['word_segments_unpacked'].astype(str)
df['word_segments_unpacked'] = df['word_segments_unpacked'].apply(lambda x: ''.join([str(i) for i in x]))
phrase_count = df[['word_segments_unpacked', 2]]


s= phrase_count.apply(lambda x: pd.Series(x['word_segments_unpacked']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'word_segments_unpacked'

phrase_count = phrase_count.drop('word_segments_unpacked', axis=1).join(s)
phrase_count = pd.DataFrame(df['word_segments_unpacked'].str.split(',').tolist(), index=df[2]).stack()

phrase_count = phrase_count.reset_index()[[0, 2]] # var1 variable is currently labeled 0
phrase_count.columns = ['word_segments_unpacked', 'ratings'] # renaming var1
phrase_count = phrase_count.reset_index(drop=False)
replace_dict_phrase_count = {'[':'',']':'','-':'','!':'','.':'',' ':'',"'":''}
for key in replace_dict_phrase_count.keys():
  phrase_count['word_segments_unpacked'] = phrase_count['word_segments_unpacked'].str.replace(key, replace_dict_phrase_count[key])
phrase_count['word_segments_unpacked'] = phrase_count['word_segments_unpacked'].str.lower()

stopwords = ['\xa0','0','1','2','3','4','5','6','7','8','9','/','$',"'d","'ll","'m",'+','maybe','from','first','here','only','put','where','got','sure','definitely','food','yet','our','go','since','really','very','two',"n't",'with','if',"'s",'which','came','all','me','(',')','makes','make','were','immediately','get','been','ahead','also','that','one','have','see','what','to','we','had','.',"'re",'it','or','he','she','we','us','how','went','no','"','of','has','by','bit','thing','place','so','ok','and','they','none','was','you',"'ve",'did','be','and','but','is','as','&','you','has','-',':','and','had','was','him','so','my','did','would','her','him','it','is','by','bit','thing','place','[',']','while','check-in','=','= =','want', 'good','husband', 'want','love','something','your','they','your','cuz','him',"i've",'her','told', 'check', 'i"m', "it's",'they', 'this','its','they','this',"don't",'the',',', 'it', 'i"ve', 'i"m', '!', '1','2','3','4', '5','6','7','8','9','0','/','.']
def filter_stopwords(text):
  for i in str(text):
    if i not in stopwords:
      return str(text)

phrase_count = phrase_count[~phrase_count['word_segments_unpacked'].isin(stopwords)]#if item in stopwords list partially matches, delete

#full matches
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='i']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='a']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='an']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='am']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='at']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='are']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='in']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='on']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='for']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='\xa0\xa0']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='\xa0']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='\(']

phrase_count_pivot = pd.pivot_table(phrase_count, index='word_segments_unpacked', columns='ratings', aggfunc='count', fill_value=0)
phrase_count_pivot.columns = [''.join(col).strip() for col in phrase_count_pivot.columns.values]#flatten index levels part 1
phrase_count_pivot = pd.DataFrame(phrase_count_pivot.to_records())#flatten index levels part 2

required_column_names = ['index1.0 star rating', 'index2.0 star rating','index3.0 star rating','index4.0 star rating','index5.0 star rating']

for i in required_column_names:
  if i not in phrase_count_pivot.columns:
    phrase_count_pivot[i] = 0
phrase_count_pivot.sample(10)

# #scale
phrase_count_pivot['index1.0 star rating'] = phrase_count_pivot['index1.0 star rating']*(-2)
phrase_count_pivot['index2.0 star rating'] = phrase_count_pivot['index2.0 star rating']*(-1)
phrase_count_pivot['index3.0 star rating'] = phrase_count_pivot['index3.0 star rating']*(-0.1)
phrase_count_pivot['index4.0 star rating'] = phrase_count_pivot['index4.0 star rating']*(1)
phrase_count_pivot['index5.0 star rating'] = phrase_count_pivot['index5.0 star rating']*(2)


phrase_count_pivot['score'] = phrase_count_pivot['index1.0 star rating'] + phrase_count_pivot['index2.0 star rating'] + phrase_count_pivot['index3.0 star rating'] + phrase_count_pivot['index4.0 star rating'] + phrase_count_pivot['index5.0 star rating']

phrase_count_pivot['score'] = phrase_count_pivot['score'].div(phrase_count_pivot['score'].max(), axis=0)
phrase_count_pivot['score'] = phrase_count_pivot['score'].round(decimals=4)
phrase_count_pivot = phrase_count_pivot.sort_values(by=('score'), ascending=False)
phrase_count_pivot.head(2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/

,word_segments_unpacked,index1.0 star rating,index5.0 star rating,index2.0 star rating,index3.0 star rating,index4.0 star rating,score
30,job,0,2,0,-0.0,0,1.0
49,small,0,2,0,-0.0,0,1.0


In [0]:
import math 
phrase_count_pivot['word_segments_unpacked'] = phrase_count_pivot['word_segments_unpacked'].str.replace('\(', '')
phrase_count_pivot['word_segments_unpacked'] = phrase_count_pivot['word_segments_unpacked'].str.replace('(', '')
phrase_count_pivot['word_segments_unpacked'] = phrase_count_pivot['word_segments_unpacked'].str.replace(')', '')
worst_terms_list = [] 
top_terms_list = []
x,y = phrase_count_pivot.shape#tuple unpacking to get the length of the dataframe
# for i in reversed(range(most_neg)):#get a fraction of the total instances
for i in reversed(range(x)):
  try:
    new_df = df[df[1].str.contains(phrase_count_pivot['word_segments_unpacked'].iloc[i])]#if word appears in review, create a dataframe
    neg_first_df = new_df.sort_values(by=2, ascending=True)#rank the dataframe with worst reviews first
    pos_first_df = new_df.sort_values(by=2, ascending=False)#rank the dataframe with most positive reviews first
    if neg_first_df[1].iloc[0] not in worst_terms_list:#get the lowest star rating review
      worst_terms_list.append(neg_first_df[1].iloc[0])#prevent duplicates
    if pos_first_df[1].iloc[0] not in top_terms_list:
      top_terms_list.append(pos_first_df[1].iloc[0])
  except IndexError as e:
    pass

In [0]:
negative_list = []
# for i in reversed(range(len(worst_terms_list))):
for i in range(-30,0):
  for list_of_words in worst_terms_list:
    word_list = list_of_words.split(' ')
    for word in word_list:
      word = word.replace(',','')
      word = word.replace('.','')
      try: 
        if phrase_count_pivot['word_segments_unpacked'].iloc[i] == word:
          try:
            index = word_list.index(word)
            string_from_phrases = ','.join(word_list[max(0,index-5):min(index+20, len(word_list))])
            replace_dict_string_from_phrases= {' ':'',',':' ',' .':'.','!':'','[':'',']':'','\n':'','1':'','2':'','3':'','4':'','5':'','6':'','7':'','8':'','9':'','0':'','/':'',"'":"'","'":''}
            for key in replace_dict_string_from_phrases.keys():
              string_from_phrases=string_from_phrases.replace(key, replace_dict_string_from_phrases[key])
            negative_list.append(string_from_phrases)
          except ValueError as e:
            pass
      except IndexError as e:
        if phrase_count_pivot['word_segments_unpacked'].iloc[0] == word:
          try:
            index = word_list.index(word)
            string_from_phrases = ','.join(word_list[max(0,index-5):min(index+20, len(word_list))])
            replace_dict_string_from_phrases= {' ':'',',':' ',' .':'.','!':'','[':'',']':'','\n':'','1':'','2':'','3':'','4':'','5':'','6':'','7':'','8':'','9':'','0':'','/':'',"'":"'","'":''}
            for key in replace_dict_string_from_phrases.keys():
              string_from_phrases=string_from_phrases.replace(key, replace_dict_string_from_phrases[key])
            negative_list.append(string_from_phrases)
          except ValueError as e:
            pass
negative_df = pd.DataFrame(negative_list)
negative_df = negative_df.reset_index(drop=False)
negative_df = negative_df.rename(columns={'index':'score', 0 : 'term'})
negative_df = negative_df.drop_duplicates(subset='term')
x,y = negative_df.shape#tuple unpacking to get the length of the dataframe
if x < 10:
  for i in range(-40,-30):
    for list_of_words in worst_terms_list:
      word_list = list_of_words.split(' ')
      for word in word_list:
        word = word.replace(',','')
        word = word.replace('.','')
        try:
          if phrase_count_pivot['word_segments_unpacked'].iloc[i] == word:
            try:
              index = word_list.index(word)
              string_from_phrases = ','.join(word_list[max(0,index-5):min(index+20, len(word_list))])
              replace_dict = {' ':'',',':' ',' .':'.','!':'','[':'',']':'','\n':'','1':'','2':'','3':'','4':'','5':'','6':'','7':'','8':'','9':'','0':'','/':'',"'":"'","'":''}
              for key in replace_dict.keys():
                string_from_phrases=string_from_phrases.replace(key, replace_dict[key])
              negative_list.append(string_from_phrases)
            except ValueError as e:
              pass
        except IndexError as e:
          if phrase_count_pivot['word_segments_unpacked'].iloc[0] == word:
            try:
              index = word_list.index(word)
              string_from_phrases = ','.join(word_list[max(0,index-5):min(index+20, len(word_list))])
              replace_dict_string_from_phrases= {' ':'',',':' ',' .':'.','!':'','[':'',']':'','\n':'','1':'','2':'','3':'','4':'','5':'','6':'','7':'','8':'','9':'','0':'','/':'',"'":"'","'":''}
              for key in replace_dict_string_from_phrases.keys():
                string_from_phrases=string_from_phrases.replace(key, replace_dict_string_from_phrases[key])
              negative_list.append(string_from_phrases)
            except ValueError as e:
              pass
negative_df_addon = pd.DataFrame(negative_list)
negative_df_addon = negative_df_addon.reset_index(drop=False)
negative_df_addon = negative_df_addon.rename(columns={'index':'score', 0 : 'term'})
negative_df = pd.concat([negative_df, negative_df_addon])
negative_df = negative_df.head(10)



In [0]:
positive_list = []
for i in range(0,30):
  for list_of_words in top_terms_list:
    word_list = list_of_words.split(' ')
    for word in word_list:
      word = word.replace(',','')
      word = word.replace('.','')
      try: 
        if phrase_count_pivot['word_segments_unpacked'].iloc[i] == word:
          try:
            index = word_list.index(word)
            string_from_phrases = ','.join(word_list[max(0,index-5):min(index+20, len(word_list))])
            replace_dict = {' ':'',',':' ',' .':'.','!':'','[':'',']':'','\n':'','1':'','2':'','3':'','4':'','5':'','6':'','7':'','8':'','9':'','0':'','/':'',"'":"'","'":''}
            for key in replace_dict.keys():
              string_from_phrases=string_from_phrases.replace(key, replace_dict[key])
            negative_list.append(string_from_phrases)
          except ValueError as e:
            pass
      except IndexError as e:
        if phrase_count_pivot['word_segments_unpacked'].iloc[0] == word:
          try:
            index = word_list.index(word)
            string_from_phrases = ','.join(word_list[max(0,index-5):min(index+20, len(word_list))])
            replace_dict = {' ':'',',':' ',' .':'.','!':'','[':'',']':'','\n':'','1':'','2':'','3':'','4':'','5':'','6':'','7':'','8':'','9':'','0':'','/':'',"'":"'","'":''}
            for key in replace_dict.keys():
              string_from_phrases=string_from_phrases.replace(key, replace_dict[key])
            negative_list.append(string_from_phrases)
          except ValueError as e:
            pass
  positive_list.append(string_from_phrases)
positive_df = pd.DataFrame(positive_list)
positive_df = positive_df.reset_index(drop=False)
positive_df = positive_df.rename(columns={'index':'score', 0 : 'term'})
positive_df = positive_df.drop_duplicates(subset='term')
x,y = positive_df.shape#tuple unpacking to get the length of the dataframe
for i in range(30,40):
  for list_of_words in top_terms_list:
    word_list = list_of_words.split(' ')
    for word in word_list:
      word = word.replace(',','')
      word = word.replace('.','')
      try:
        if phrase_count_pivot['word_segments_unpacked'].iloc[i] == word:
          try:
            index = word_list.index(word)
            string_from_phrases = ','.join(word_list[max(0,index-5):min(index+20, len(word_list))])
            replace_dict = {' ':'',',':' ',' .':'.','!':'','[':'',']':'','\n':'','1':'','2':'','3':'','4':'','5':'','6':'','7':'','8':'','9':'','0':'','/':'',"'":"'","'":''}
            for key in replace_dict.keys():
              string_from_phrases=string_from_phrases.replace(key, replace_dict[key])
            negative_list.append(string_from_phrases)
          except ValueError as e:
            pass
      except IndexError as e:
        if phrase_count_pivot['word_segments_unpacked'].iloc[0] == word:
          try:
            index = word_list.index(word)
            string_from_phrases = ','.join(word_list[max(0,index-5):min(index+20, len(word_list))])
            replace_dict = {' ':'',',':' ',' .':'.','!':'','[':'',']':'','\n':'','1':'','2':'','3':'','4':'','5':'','6':'','7':'','8':'','9':'','0':'','/':'',"'":"'","'":''}
            for key in replace_dict.keys():
              string_from_phrases=string_from_phrases.replace(key, replace_dict[key])
            negative_list.append(string_from_phrases)
          except ValueError as e:
            pass
positive_df_addon = pd.DataFrame(negative_list)
positive_df_addon = positive_df_addon.reset_index(drop=False)
positive_df_addon = positive_df_addon.rename(columns={'index':'score', 0 : 'term'})
positive_df = pd.concat([positive_df, positive_df_addon])
positive_df = positive_df.head(10)

In [9]:
results = {'positive': [{'term': pos_term, 'score': pos_score} for pos_term, pos_score in zip(positive_df['term'], positive_df['score'])], 'negative': [{'term': neg_term, 'score': neg_score} for neg_term, neg_score in zip(negative_df['term'], negative_df['score'])]}
results

{'negative': [{'score': 0,
   'term': 'Can I give them zero stars? So I reached out to Israel (owner) during the st week of October for a move on November st'},
  {'score': 1,
   'term': 'cant move me at the quoterate he had agreed to earlier. Since the moving date was three days away  I agreed to an additional $'},
  {'score': 2,
   'term': 'them zero stars? So I reached out to Israel (owner) during the st week of October for a move on November st (yep today) Was'},
  {'score': 3,
   'term': 'agreed to earlier. Since the moving date was three days away  I agreed to an additional $ an hour quote. '},
  {'score': 4,
   'term': 'to earlier. Since the moving date was three days away  I agreed to an additional $ an hour quote. '},
  {'score': 5,
   'term': 'today) Was asked to pay $ deposit to confirm the move which I did right away. I called Israel again on Oct th to reconfirm.'},
  {'score': 6,
   'term': 'called Israel again on Oct th to reconfirm. He told me that he cant move me at the